# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [1]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = "/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
status_req = requests.get(root_url + status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if status_req.status_code == 200:
    print(status_req.text)
else:
    print(status_req.status_code)


"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [2]:
# Set the countries_url variable (1 line)
# countries_url = "https://country-leaders.onrender.com/status/countries"
countries_url = "/countries"
# query the /countries endpoint using the get() method and store it in the req variable (1 line)
countries_req = requests.get(root_url + countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = countries_req.json()

# display the request's status code and the countries variable (1 line)
print(countries_req.status_code, countries)


403 {'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [3]:
# Set the cookie_url variable (1 line)
cookie_url = "/cookie"

# Query the enpoint, set the cookies variable and display it (2 lines)
cookies_req = requests.get(root_url + cookie_url).cookies
print(cookies_req.values())


['cc6b9ea5-116b-44d2-bac0-a20ee4ed03fc']


Try to query the countries endpoint using the cookie, save the output and print it.

In [4]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(root_url + countries_url, cookies=cookies_req).json()

# display the countries variable (1 line)
print(countries)

['be', 'us', 'ru', 'fr', 'ma']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [5]:
# Set the leaders_url variable (1 line)
leaders_url = "/leaders"

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders_req = requests.get(root_url + leaders_url, cookies=cookies_req).json()

# display the leaders variable (1 line)
print(leaders_req)


{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [6]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)
leaders_req = requests.get(root_url + leaders_url, params={"country": "fr"}, cookies=cookies_req).json()
# display the leaders variable (1 line)
print(leaders_req)

[{'id': 'Q157', 'first_name': 'François', 'last_name': 'Hollande', 'birth_date': '1954-08-12', 'death_date': None, 'place_of_birth': 'Rouen', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Hollande', 'start_mandate': '2012-05-15', 'end_mandate': '2017-05-14'}, {'id': 'Q329', 'first_name': 'Nicolas', 'last_name': 'Sarkozy', 'birth_date': '1955-01-28', 'death_date': None, 'place_of_birth': 'Paris', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Nicolas_Sarkozy', 'start_mandate': '2007-05-16', 'end_mandate': '2012-05-15'}, {'id': 'Q2038', 'first_name': 'François', 'last_name': 'Mitterrand', 'birth_date': '1916-10-26', 'death_date': '1996-01-08', 'place_of_birth': 'Jarnac', 'wikipedia_url': 'https://fr.wikipedia.org/wiki/Fran%C3%A7ois_Mitterrand', 'start_mandate': '1981-05-21', 'end_mandate': '1995-05-17'}, {'id': 'Q2042', 'first_name': 'Charles', 'last_name': 'de Gaulle', 'birth_date': '1890-11-22', 'death_date': '1970-11-09', 'place_of_birth': 'Lille', 'wikipedia_url': 'h

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [7]:
# 4 lines
leaders_per_country = {}
for country in countries:
    leaders_per_country[country] = requests.get((f"{root_url}/leaders"), params={"country": country}, cookies = cookies_req).json()
leaders_per_country

{'be': [{'id': 'Q12978',
   'first_name': 'Guy',
   'last_name': 'Verhofstadt',
   'birth_date': '1953-04-11',
   'death_date': None,
   'place_of_birth': 'Dendermonde',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
   'start_mandate': '1999-07-12',
   'end_mandate': '2008-03-20'},
  {'id': 'Q12981',
   'first_name': 'Yves',
   'last_name': 'Leterme',
   'birth_date': '1960-10-06',
   'death_date': None,
   'place_of_birth': 'Wervik',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
   'start_mandate': '2009-11-25',
   'end_mandate': '2011-12-06'},
  {'id': 'Q12983',
   'first_name': 'Herman',
   'last_name': 'None',
   'birth_date': '1947-10-31',
   'death_date': None,
   'place_of_birth': 'Etterbeek',
   'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy',
   'start_mandate': '2008-12-30',
   'end_mandate': '2009-11-25'},
  {'id': 'Q14989',
   'first_name': 'Léon',
   'last_name': 'Delacroix',
   'birth_date': '1867-12-27',
   'd

In [8]:
# or 1 line

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [9]:
# < 15 lines
def get_leaders():

    leaders_per_country = {}

    # Defining the URLs
    root_url = "https://country-leaders.onrender.com"

    # Getting the cookies
    cookie_url = "/cookie"
    cookies_req = requests.get(root_url + cookie_url).cookies

    # Getting the countries
    countries_url = "/countries"
    countries_req = requests.get(root_url + countries_url, cookies=cookies_req).json()

    # To ensure that all requests will share the same session, 
    # therefore the same cookies.
   
    session = requests.Session()

    for country in countries:
        leaders_per_country[country] = session.get(f"{root_url}/leaders", params={"country": country}, cookies= cookies_req).json()
    return leaders_per_country

print(get_leaders())

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [10]:
# 2 lines
leaders_per_country = get_leaders() 
print(leaders_per_country)  

{'be': [{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'htt

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [11]:
# 3 lines

wikipedia_url = "https://nl.wikipedia.org/wiki/Yves_Leterme"

# User-Agent identifies your request as coming from a browser, 
# so the website treats it like a normal visitor instead of blocking it as a bot.
leader = requests.get(wikipedia_url, headers={"User-Agent": "Mozilla/5.0"}) 
print(leader.text)


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="nl" dir="ltr">
<head>
<meta charset="UTF-8">
<title>Yves Leterme - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-featur

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [12]:
# 3 lines
from bs4 import BeautifulSoup
    
# Suppose you already have the HTML text of a Wikipedia page
html_text = leader.text

# Load it into BeautifulSoup
soup = BeautifulSoup(html_text, "html.parser")

# Prettify and print it to see the structure
print(soup.prettify()) 


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="nl">
 <head>
  <meta charset="utf-8"/>
  <title>
   Yves Leterme - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-en

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [13]:
# 2 lines
paragraphs = soup.find_all("p")
print(paragraphs)

[<p><b>Yves Camille Désiré Leterme</b> (<a href="/wiki/Wervik" title="Wervik">Wervik</a>, <a href="/wiki/6_oktober" title="6 oktober">6 oktober</a> <a href="/wiki/1960" title="1960">1960</a>) is een voormalig <a class="mw-redirect" href="/wiki/Belgi%C3%AB_(hoofdbetekenis)" title="België (hoofdbetekenis)">Belgisch</a> <a href="/wiki/Politicus" title="Politicus">politicus</a>. Hij was onder meer <a href="/wiki/Minister-president_van_Vlaanderen" title="Minister-president van Vlaanderen">Vlaams minister-president</a> en <a href="/wiki/Premier_van_Belgi%C3%AB" title="Premier van België">premier van België</a>. Hij was lang boegbeeld van de <a href="/wiki/Christen-Democratisch_en_Vlaams" title="Christen-Democratisch en Vlaams">CD&amp;V</a> en boegbeeld van het toenmalige kartel <a href="/wiki/Valentijnskartel" title="Valentijnskartel">CD&amp;V/N-VA</a>.
</p>, <p>Hij werd geboren in <a href="/wiki/Wervik" title="Wervik">Wervik</a>, op de grens tussen <a href="/wiki/Frankrijk" title="Frankrijk

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [14]:
# <10 lines

for paragraph in paragraphs:
    # Finds <b> and checks if it starts with element <p>. 
    if paragraph.find("b") == paragraph.contents[0]:
        first_paragraph = paragraph.get_text(strip=True) # I used strip, to make the text cleaner.
        break 
print(first_paragraph)
    


Yves Camille Désiré Leterme(Wervik,6 oktober1960) is een voormaligBelgischpoliticus. Hij was onder meerVlaams minister-presidentenpremier van België. Hij was lang boegbeeld van deCD&Ven boegbeeld van het toenmalige kartelCD&V/N-VA.


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [15]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

def get_first_paragraph(wikipedia_url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)..."}
    print(wikipedia_url)
    html_text = requests.get(wikipedia_url, headers = headers).text
    soup = BeautifulSoup(html_text, "html.parser")
    for paragraph in soup.find_all("p"):
    # Finds <b> and checks if it starts with element <p>. 
        if paragraph.find("b") == paragraph.contents[0]:
            first_paragraph = paragraph.get_text(strip=True) # I used strip, to make the text cleaner.
            return first_paragraph

         


In [16]:
# Test: 3 lines
first_paragraph = get_first_paragraph("https://nl.wikipedia.org/wiki/Herman_Van_Rompuy")
print(first_paragraph)

https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
Herman Achille graaf Van Rompuy(Etterbeek,31 oktober1947) is eenBelgischepoliticus van de christendemocratische partijCD&V. Hij was tussen 30 december 2008 en 25 november 2009 depremier van Belgiëen van 1 januari 2010[2]tot en met 30 november 2014voorzitter van de Europese Raad, in de media vaak incorrect 'president van Europa' genoemd.[3]Op 19 november 2009 werd bekendgemaakt dat hij was verkozen tot de eerste permanente voorzitter en op 1 maart 2012 werd Van Rompuy met algemene stemmen door de 27 staatshoofden en regeringsleiders van de EU voor een tweede mandaat benoemd. Na deze termijn was hij niet meer herkiesbaar.


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [17]:
# 3 lines
import re
test_urls = "https://nl.wikipedia.org/wiki/Herman_Van_Rompuy"
cleaned_urls = re.sub(r'\?.*?(?=\s|$)', '', test_urls)  # removes everything from '?' to the next space or end
print(cleaned_urls)


https://nl.wikipedia.org/wiki/Herman_Van_Rompuy


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [18]:
# 10 lines

def get_first_paragraph(wikipedia_url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)..."}
    # print(wikipedia_url)
    html_text = requests.get(wikipedia_url, headers = headers).text
    soup = BeautifulSoup(html_text, "html.parser")
    for paragraph in soup.find_all("p"):
    # Finds <b> and checks if it starts with element <p>. 
        if paragraph.find("b") == paragraph.contents[0]:
            first_paragraph = paragraph.get_text(strip=True) # I used strip, to make the text cleaner.
            first_paragraph = re.sub(r'\[\d+\]', '', first_paragraph) # <- Added a line here
            return first_paragraph
print (first_paragraph)


Herman Achille graaf Van Rompuy(Etterbeek,31 oktober1947) is eenBelgischepoliticus van de christendemocratische partijCD&V. Hij was tussen 30 december 2008 en 25 november 2009 depremier van Belgiëen van 1 januari 2010[2]tot en met 30 november 2014voorzitter van de Europese Raad, in de media vaak incorrect 'president van Europa' genoemd.[3]Op 19 november 2009 werd bekendgemaakt dat hij was verkozen tot de eerste permanente voorzitter en op 1 maart 2012 werd Van Rompuy met algemene stemmen door de 27 staatshoofden en regeringsleiders van de EU voor een tweede mandaat benoemd. Na deze termijn was hij niet meer herkiesbaar.


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [19]:
# < 20 lines


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [20]:
# < 20 lines

def get_leaders():
    leaders_per_country = {}
    # Defining the URLs
    root_url = "https://country-leaders.onrender.com"
    # Getting the cookies
    cookie_url = "/cookie"
    cookies_req = requests.get(root_url + cookie_url).cookies
    
    # Getting the countries
    countries_url = "/countries"
    countries_req = requests.get(root_url + countries_url, cookies=cookies_req).json()

    # To ensure that all requests will share the same session, 
    # therefore the same cookies.
    session = requests.Session()
    for country in countries_req:
        
        # so since my cookie expires in the middle of then run, 
        # I could incorporate an extra cookie inside the loop (code below)
        # cookies_req1 = requests.get(root_url + cookie_url).cookies 
        # rename the cookies_req to cookies_req1 (if i will use the code above)
        leaders_per_country[country] = session.get(f"{root_url}/leaders", params={"country": country}, cookies= cookies_req).json()
        for leader in leaders_per_country[country]:
            # print(leader)
            url = leader["wikipedia_url"]
            first_paragraph = get_first_paragraph(url)
            leader["first_paragraph"] = first_paragraph

    return leaders_per_country
       


In [21]:
# Check the output of your function (2 lines)
generated_leaders = get_leaders()
print(generated_leaders)





TypeError: string indices must be integers, not 'str'

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [22]:
# < 25 lines

def get_leaders():
    leaders_per_country = {}
    # Defining the URLs
    root_url = "https://country-leaders.onrender.com"
    # Getting the cookies
    cookie_url = "/cookie"
    cookies_req = requests.get(root_url + cookie_url).cookies
    
    # Getting the countries
    countries_url = "/countries"
    # query the /countries endpoint using the get() method and store it in the req variable (1 line)
    countries_req = requests.get(root_url + countries_url, cookies=cookies_req).json()

    # To ensure that all requests will share the same session, 
    # therefore the same cookies.
    session = requests.Session()
    for country in countries_req:
        try: 
            leaders_per_country[country] = session.get(f"{root_url}/leaders", params={"country": country}, cookies= cookies_req).json()
            for leader in leaders_per_country[country]:
                # print(leader)
                url = leader["wikipedia_url"]
                first_paragraph = get_first_paragraph(url)
                leader["first_paragraph"] = first_paragraph
        except Exception as exc:
            # new cookie + continue
            print(f"Error cookie has expired. Fetching leaders for {country}, exception= {exc}")
            cookies_req = requests.get(root_url + cookie_url).cookies  # new cookie
            continue
        
    return leaders_per_country


Check the output of your function again.

In [23]:
# Check the output of your function (1 line)
print(get_leaders())

KeyboardInterrupt: 

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# < 20 lines
from requests import Session
from bs4 import BeautifulSoup
import re

def get_first_paragraph(wikipedia_url, session):
    # User-Agent identifies your request as coming from a browser, 
    # so the website treats it like a normal visitor instead of blocking it as a bot.
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)..."}
    # print(wikipedia_url)
    html_text = session.get(wikipedia_url, headers = headers).text
    soup = BeautifulSoup(html_text, "html.parser")
    for paragraph in soup.find_all("p"):
    # Finds <b> and checks if it starts with element <p>. 
        if paragraph.find("b") == paragraph.contents[0]:
            first_paragraph = paragraph.get_text(strip=True) # I used strip, to make the text cleaner.
            first_paragraph = re.sub(r'\[\d+\]', '', first_paragraph) # <- Added a line here
            return first_paragraph


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# <25 lines
import requests

def get_leaders():
    leaders_per_country = {}
    # Defining the URLs
    root_url = "https://country-leaders.onrender.com"
    # Getting the cookies
    cookie_url = "/cookie"
    cookies_req = requests.get(root_url + cookie_url).cookies
    
    # Getting the countries
    countries_url = "/countries"
    # query the /countries endpoint using the get() method and store it in the req variable (1 line)
    countries_req = requests.get(root_url + countries_url, cookies=cookies_req).json()

    # To ensure that all requests will share the same session, 
    # therefore the same cookies.
    session = requests.Session()
    for country in countries_req:
        try: 
            leaders_per_country[country] = session.get(f"{root_url}/leaders", params={"country": country}, cookies= cookies_req).json()
            for leader in leaders_per_country[country]:
                # print(leader) to see what i get
                url = leader["wikipedia_url"]
                first_paragraph = get_first_paragraph(url, session)
                leader["first_paragraph"] = first_paragraph
        except Exception as exc:
            # new cookie + continue
            print(f"Error fetching leaders for {country}, exception= {exc}")
            cookies_req = requests.get(root_url + cookie_url).cookies  # new cookie
            continue
        
    return leaders_per_country

Test your new functions.



In [ ]:
import requests
session = requests.Session()
first_par = get_first_paragraph(wikipedia_url, session)
print(first_par)
lead = get_leaders()
print(lead)

Yves Camille Désiré Leterme(Wervik,6 oktober1960) is een voormaligBelgischpoliticus. Hij was onder meerVlaams minister-presidentenpremier van België. Hij was lang boegbeeld van deCD&Ven boegbeeld van het toenmalige kartelCD&V/N-VA.
Error fetching leaders for ru, exception= string indices must be integers, not 'str'


KeyboardInterrupt: 

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines
import json
leaders = get_leaders()

with open("leaders.json", "w", encoding="utf-8") as f:
    json.dumps(leaders, f, ensure_ascii=False, indent=2)


https://nl.wikipedia.org/wiki/Guy_Verhofstadt
https://nl.wikipedia.org/wiki/Yves_Leterme
https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix
https://nl.wikipedia.org/wiki/Henri_Carton_de_Wiart
https://nl.wikipedia.org/wiki/Georges_Theunis
https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere
https://nl.wikipedia.org/wiki/Prosper_Poullet
https://nl.wikipedia.org/wiki/Henri_Jaspar
https://nl.wikipedia.org/wiki/Jules_Renkin
https://nl.wikipedia.org/wiki/Paul_van_Zeeland
https://nl.wikipedia.org/wiki/Achiel_Van_Acker
https://nl.wikipedia.org/wiki/Camille_Huysmans
https://nl.wikipedia.org/wiki/Gaston_Eyskens
https://nl.wikipedia.org/wiki/Leo_Tindemans
https://nl.wikipedia.org/wiki/Elio_Di_Rupo
https://nl.wikipedia.org/wiki/Jean-Luc_Dehaene
https://nl.wikipedia.org/wiki/Frans_Schollaert
https://nl.wikipedia.org/wiki/Pieter_de_Decker
https://nl.wikipedia.org/wiki/Paul-Henri_Spaak
https://nl.wikipedia.org/wiki/Auguste_Beernaert
https://nl.wikipedia.org

Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [ ]:
# 3 lines
with open("leaders.json", "r", encoding="utf-8") as f:
    leaders_loaded = json.load(f)
print("Are they equal?", leaders == leaders_loaded)

NameError: name 'json' is not defined

Make a function `save(leaders_per_country)` to call this code easily.

In [ ]:
# 3 lines
def save(leaders_per_country):
    with open("leaders.json", "w", encoding="utf-8") as f:
        json.dump(leaders_per_country, f, ensure_ascii=False, indent=2)


In [ ]:
# Call the function (1 line)
save(get_leaders())


https://nl.wikipedia.org/wiki/Guy_Verhofstadt
https://nl.wikipedia.org/wiki/Yves_Leterme
https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix
https://nl.wikipedia.org/wiki/Henri_Carton_de_Wiart
https://nl.wikipedia.org/wiki/Georges_Theunis
https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere
https://nl.wikipedia.org/wiki/Prosper_Poullet
https://nl.wikipedia.org/wiki/Henri_Jaspar
https://nl.wikipedia.org/wiki/Jules_Renkin
https://nl.wikipedia.org/wiki/Paul_van_Zeeland
https://nl.wikipedia.org/wiki/Achiel_Van_Acker
https://nl.wikipedia.org/wiki/Camille_Huysmans
https://nl.wikipedia.org/wiki/Gaston_Eyskens
https://nl.wikipedia.org/wiki/Leo_Tindemans
https://nl.wikipedia.org/wiki/Elio_Di_Rupo
https://nl.wikipedia.org/wiki/Jean-Luc_Dehaene
https://nl.wikipedia.org/wiki/Frans_Schollaert
https://nl.wikipedia.org/wiki/Pieter_de_Decker
https://nl.wikipedia.org/wiki/Paul-Henri_Spaak
https://nl.wikipedia.org/wiki/Auguste_Beernaert
https://nl.wikipedia.org

## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!